In [ ]:
import keras
from keras.datasets import fashion_mnist
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential,Model
from keras.layers import Dense,Conv2D,Flatten,MaxPooling2D,GlobalAveragePooling2D
from keras.utils import plot_model

In [ ]:
(X_train,Y_train),(X_test,Y_test)  = fashion_mnist.load_data()

In [ ]:
X_train = X_train.reshape(60000,28,28,1)
X_test = X_test.reshape(10000,28,28,1)

X_train = X_train/255
X_test  = X_test/255
X_train = X_train.astype('float')
X_test  = X_test.astype('float')


In [ ]:
def show_img(img):
    img  = np.array(img,dtype='float')
    img = img.reshape((28,28))
    plt.imshow(img)


show_img(X_train[1])

In [ ]:
model = Sequential()
model.add(Conv2D(16,input_shape=(28,28,1),kernel_size=(3,3),activation='relu',padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32,kernel_size=(3,3),activation='relu',padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,kernel_size=(3,3),activation='relu',padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128,kernel_size=(3,3),activation='relu',padding='same'))
model.add(GlobalAveragePooling2D())
model.add(Dense(10,activation='softmax'))

model.summary()
model.compile(loss='sparse_categorical_crossentropy',metrics=['accuracy'],optimizer='adam')
model.fit(X_train,Y_train,batch_size=32,epochs=5,validation_split=0.1,shuffle=True)

In [ ]:
print(model.layers[-3].name)
print(model.layers[-1].name)

In [ ]:
from keras.models import Model
import scipy as sp
gap_weights = model.layers[-1].get_weights()[0]
gap_weights.shape

cam_model  = Model(inputs=model.input,outputs=(model.layers[-3].output,model.layers[-1].output))
cam_model.summary()

In [ ]:
features,results = cam_model.predict(X_test)
features.shape

In [ ]:
def show_cam(i):
  image_index = i
  features_for_img = features[image_index,:,:,:]
  prediction = np.argmax(results[image_index])
  class_activation_weights = gap_weights[:,prediction]
  class_activation_features = sp.ndimage.zoom(features_for_img, (28/3, 28/3, 1), order=2)
  cam_output  = np.dot(class_activation_features,class_activation_weights)
  print('Predicted Class = ' +str(prediction)+ ', Probability = ' + str(results[image_index][prediction]))
  plt.imshow(np.squeeze(X_test[image_index],-1), alpha=0.5)
  if results[image_index][prediction]>0.95:
    cmap_str = 'Greens'
  else:
    cmap_str = 'Reds'
  plt.imshow(cam_output, cmap=cmap_str, alpha=0.5)
  plt.show()


desired_class=1
for i in range(0,100):
  if np.argmax(results[i]) == desired_class:
    show_cam(i)